In [13]:

#  This one is for Elastic search the previous code is for Min Search
from elasticsearch.helpers import bulk
import json
import glob
from elasticsearch import Elasticsearch
from groq import Groq

client = Groq(api_key="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")

# Connect to Elasticsearch
es = Elasticsearch(
    hosts=[{
        'host': 'localhost',
        'port': 9200,
        'scheme': 'http'  # Specify the scheme
    }]
)


# Index name
index_name = 'nbe_directives'

# Define the index settings and mappings
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "page_number": {"type": "text"},
            "content": {"type": "text"},
            "source_file": {"type": "text"}
        }
    }
}

# Delete the index if it exists
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Create the index
es.indices.create(index=index_name, body=index_settings)

# Initialize an empty list to hold the documents
documents = []

# Specify the correct path to the directory containing the JSON files
json_files_path = 'C:/Users/EstifanosT/AwashRagSystem/ConvertedToJson/*.json'

# Get the list of JSON files in the directory
json_files = glob.glob(json_files_path)

# Iterate over each JSON file in the directory
for json_file in json_files:
    try:
        with open(json_file, 'rt', encoding='utf-8') as f_in:
            doc_raw = json.load(f_in)
            # Check if 'pages' key exists in the JSON
            if 'pages' not in doc_raw:
                continue
            # Iterate over each page dictionary in the 'pages' list
            for page_dict in doc_raw['pages']:
                content = page_dict.get('content', '').strip()
                if content:  # Ensure the content is not empty
                    # Create a new dictionary to store the content and page number
                    doc = {
                        '_index': index_name,
                        '_source': {
                            'page_number': page_dict.get('page_number', 'Unknown'),
                            'content': content,
                            'source_file': json_file
                        }
                    }
                    # Append the dictionary to the 'documents' list
                    documents.append(doc)
    except Exception as e:
        print(f"Error processing file {json_file}: {e}")

if not documents:
    raise ValueError("No valid documents found for indexing. Ensure the documents have content.")

# Bulk index the documents
bulk(es, documents)

def search(query):
    # Search the Elasticsearch index
    results = es.search(
        index=index_name,
        body={
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["content", "source_file"]
                }
            }
        }
    )

    # Extract the relevant information from the search results
    hits = results['hits']['hits']
    search_results = []
    for hit in hits:
        search_results.append(hit['_source'])

    return search_results

def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Provide a direct answer to the QUESTION based on the CONTEXT from the NBE directive.
    Answer the QUESTION concisely and accurately using only the facts from the CONTEXT.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context += f"source_file: {doc['source_file']}\npage_number: {doc['page_number']}\ncontent: {doc['content']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


def llm(prompt):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )
    return response.choices[0].message.content

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer


In [14]:
rag('What does it mean Acceptable Foreign Currency?')

"I'm happy to help!"

In [15]:
rag('what does it mean by Concession?')

'Based on the provided context, specifically from the sources:\n\n* source_file: C:/Users/EstifanosT/AwashRagSystem/ConvertedToJson\\FCP-01-2020.json\n* page_number: 98\n* content: ... WARNING: Fees other than those listed above may apply to your account. ...\n\nA concession refers to:\n\n"... the lender(s) of the borrower, for economic or contractual reasons relating to the borrower\'s financial difficulty, having granted to the borrower a concession(s) that the lender(s) would not otherwise consider..."\n\nIn simpler terms, a concession is a favor or relaxation in credit terms that a lender grants to a borrower experiencing financial difficulties, which the lender would not normally do. This can include, for example, releasing collateral or accepting lower levels of collateralization, allowing the conversion of debt to equity, deferring recovery/collection actions for extended periods of time, or easing of covenants.'